In [1]:
import sys

In [2]:
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install gym==0.25.1
!{sys.executable} -m pip install gym[classic_control]==0.25.1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython import display
import time
import math
import random

from timeit import default_timer as timer
%matplotlib inline

In [31]:
env = gym.make("CartPole-v1")
print(env.action_space.n)

2


/home/sam/.local/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/sam/.local/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [32]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0
prior_reward = 0

Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1

epsilon_decay_value = 0.99995

q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

def get_discrete_state(state):
    discrete_state = state/np_array_win_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(np.int))


for episode in range(EPISODES + 1):
    t0 = time.time() # set the initial time
    discrete_state = get_discrete_state(env.reset()) # get the discrete start for the restarted environment
    done = False
    episode_reward = 0 # reward starts as 0 for each episode

    if episode % 2000 == 0:
        print("Episode: " + str(episode))

    while not done:

        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) # take cordinated action
        else:

            action = np.random.randint(0, env.action_space.n) # do a random ation

        new_state, reward, done, _ = env.step(action) # step action to get new states, reward, and the "done" status.

        episode_reward += reward # add the reward

        new_discrete_state = get_discrete_state(new_state)

        if episode % 2000 == 0:
            #render
            env.render()

        if not done:
            #update q-table
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + \
                LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.05:
        #epsilon modification
        if episode_reward > prior_reward and episode > 10000:
            epsilon = math.pow(epsilon_decay_value, episode - 10000)

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() # episode has finished
    episode_total = t1 - t0 # episode total time
    total = total + episode_total

    total_reward += episode_reward # episode total reward
    prior_reward = episode_reward

    if episode % 1000 == 0:
        #every 1000 episodes print the average time and the average reward
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()


/tmp/ipykernel_95270/1817636120.py:21: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(discrete_state.astype(np.int))
/home/sam/.local/lib/python3.10/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render 

Episode: 0
Time Average: 0.0007596485614776612
Mean Reward: 0.015
Time Average: 0.0006046288013458252
Mean Reward: 22.068
Episode: 2000
Time Average: 0.00127907395362854
Mean Reward: 22.504
Time Average: 0.0006143548488616944
Mean Reward: 22.314
Episode: 4000
Time Average: 0.0007833573818206787
Mean Reward: 21.801
Time Average: 0.0005968797206878662
Mean Reward: 21.81
Episode: 6000
Time Average: 0.0010742180347442626
Mean Reward: 22.745
Time Average: 0.0006068096160888671
Mean Reward: 22.201
Episode: 8000
Time Average: 0.0010612168312072753
Mean Reward: 22.008
Time Average: 0.0006324729919433594
Mean Reward: 22.53
Episode: 10000
Time Average: 0.0009132781028747558
Mean Reward: 21.342
Epsilon: 0.9753093024395111
Epsilon: 0.9512282354250458
Time Average: 0.0006210019588470459
Mean Reward: 22.511
Epsilon: 0.9277417467531685
Episode: 12000
Time Average: 0.0008816008567810059
Mean Reward: 24.537
Epsilon: 0.8607047486686201
Time Average: 0.0007172863483428955
Mean Reward: 25.728
Episode: 140